In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src import layer, losses
from src.model import Model

In [ ]:
data_df = pd.read_csv('ionosphere_data.csv', header=None)

data_df['is_good'] = data_df[34].apply(lambda x: 1 if x == 'g' else 0)
data_df = data_df.drop(34, axis=1)

data_df


In [ ]:
dataset = np.array(data_df)
np.random.shuffle(dataset)

train_ratio = 0.8
train_size = int(len(dataset) * train_ratio)

train_feature = dataset[:train_size, :-1]
test_feature = dataset[train_size:, :-1]
train_target = dataset[:train_size, -1:]
test_target = dataset[train_size:, -1:]

In [ ]:
def split_batch(x, y, batch_size):
    for i in range(0, len(x), batch_size):
        if i + batch_size < len(x):
            yield x[i:i + batch_size], y[i:i + batch_size]
        yield x[i:], y[i:]

In [ ]:
INPUT_FEATURES = train_feature.shape[1]

model = Model([
    layer.Linear(INPUT_FEATURES, 8),
    layer.ReLU(),
    layer.Linear(8, 1),
    layer.Sigmoid(),
])

bce = losses.BinaryCrossEntropy()

In [ ]:
EPOCHES = 1000
BATCH_SIZE = 128

train_losses = []
for epoch in range(EPOCHES):

    losses = []
    for x, y in split_batch(train_feature, train_target, BATCH_SIZE):
        y_pred = model(x)
        losses.append(bce(y_pred, y))
        grad = bce.backward()
        model.backward(grad)
        model.update(0.001)

    train_losses.append(np.mean(losses))

    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch}: {np.mean(losses)}')

In [ ]:
plt.plot(train_losses)

In [ ]:
y_pred = model(test_feature)
print(bce(y_pred, test_target))

In [ ]:
pd.DataFrame(np.concatenate([y_pred, test_target], axis=1))